# GUÍA BRAZO ROBÓTICO EN GAZEBO


## Iniciando la simulacion de Gazebo

Para simular brazo robótico UR10 junto con el gripper en Gazebo, debes seguir estos pasos:

1. Abre una Nueva Terminal: Primero, necesitas abrir una nueva ventana de terminal en tu sistema (Ctrl + Alt + T).

2. Ejecuta el Comando de Lanzamiento: En la terminal recién abierta, ejecuta el siguiente comando:


``` shellscript
user@pc:~$ roslaunch manipulacion_pkg  robot_gazebo.launch tipo_gripper:=jaco \
            objeto:=mustard_bottle

```

## Ejemplo: Control de un Brazo Robótico en Gazebo

En este ejemplo, exploraremos cómo leer las posiciones actuales de las articulaciones de un brazo robótico en Gazebo y cómo enviar comandos para mover el brazo a nuevas posiciones. Utilizaremos la clase `GazeboRobot`, que hereda de `Robot`, y proporciona funcionalidades específicas para interactuar con el brazo robótico en la simulación de Gazebo.

1. **Inicializar el Nodo ROS**: Primero, inicializamos un nodo ROS. Este nodo nos facilitará la comunicación con el sistema ROS y permitirá el control del brazo robótico en la simulación de Gazebo.

2. **Creación de una Instancia de GazeboRobot**: Se crea una instancia de `GazeboRobot`, especificando los nombres de las articulaciones del brazo robótico que queremos controlar. Esto nos permite interactuar específicamente con nuestro brazo robótico en Gazebo.

3. **Lectura de Posiciones Actuales de las Articulaciones**: Utilizamos el método `obtener_posiciones_articulaciones` para leer las posiciones actuales de las articulaciones del brazo robótico. 

4. **Envío de Comando a una Posición articular deseada**:  Para mover el brazo robótico a una posición deseada, usamos el método `command_posicion_articulaciones`, especificando las posiciones deseadas para las articulaciones (`posicion_deseada`) y el tiempo desde que se envía el comando hasta alcanzar estas posiciones (`time_from_start`).

5. **Envío de Comando de un path de Posiciones articulares**: Utilizamos el método `command_path_posicion_articulaciones` para definir una trayectoria de posiciones articulares. Aquí, `path` representa una lista de configuraciones articulares que el brazo debe alcanzar de manera secuencial. `time_between_points` indica el tiempo en segundos necesario para que el brazo se mueva de una configuración a la siguiente dentro de la trayectoria. Por último, `start_time` define el tiempo en segundos desde el envío del comando hasta que el brazo llega a la primera posición de la trayectoria.





In [1]:
import rospy
import manipulacion_lib  

# 1. Inicializar el Nodo ROS
if not rospy.get_node_uri():
    rospy.init_node('robot_arm_controller', anonymous=True)

# 2. Creación de una Instancia de `GazeboRobot`
gazebo_robot = manipulacion_lib.GazeboRobot(nombres_articulaciones=[
    'shoulder_pan_joint',
    'shoulder_lift_joint',
    'elbow_joint',
    'wrist_1_joint',
    'wrist_2_joint',
    'wrist_3_joint'
])

# 3. Lectura de Posiciones Actuales de las Articulaciones
posiciones_actuales = gazebo_robot.obtener_posiciones_articulaciones()
print("Posiciones actuales de las articulaciones:", posiciones_actuales)

# 4. Envío de Comando a una Posición Específica
posicion_deseada = [0.5, -1.0, 1.0, -1.0, 0.0, 1.5]
# Define la posición deseada para las articulaciones y el tiempo para alcanzarla
gazebo_robot.command_posicion_articulaciones(posicion_deseada, time_from_start=2)


Posiciones actuales de las articulaciones: [0.4999543653957419, -0.49427635610777365, 0.5018147340750101, -0.4988525580954519, 0.6000029376823504, 0.49150842768728165]


Unknown tag "surface" in /robot[@name='ur10']/link[@name='hand_6_hand_limb']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_0_pos_transmission']/actuator[@name='hand_finger_joint_0_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_2_pos_transmission']/actuator[@name='hand_finger_joint_2_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_joint_4_pos_transmission']/actuator[@name='hand_finger_joint_4_effort_motor']


In [2]:
# 5. Envío de Comando path the posiciones articulares
path = [
    [0.0, -0.8, 0.0, -0.3, 0.0, 0.5],
    [0.0, -1.0, 1.0, -1.0, 0.0, 1.0],
    [0.5, -0.5, 0.5, -0.5, 0.6, 0.5]
]
gazebo_robot.command_path_posicion_articulaciones(path, time_between_points=2, start_time=2)


header: 
  seq: 0
  stamp: 
    secs: 3516
    nsecs: 699000000
  frame_id: ''
joint_names: 
  - shoulder_pan_joint
  - shoulder_lift_joint
  - elbow_joint
  - wrist_1_joint
  - wrist_2_joint
  - wrist_3_joint
points: 
  - 
    positions: [0.0, -0.8, 0.0, -0.3, 0.0, 0.5]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 2
      nsecs:         0
  - 
    positions: [0.0, -1.0, 1.0, -1.0, 0.0, 1.0]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 4
      nsecs:         0
  - 
    positions: [0.5, -0.5, 0.5, -0.5, 0.6, 0.5]
    velocities: []
    accelerations: []
    effort: []
    time_from_start: 
      secs: 6
      nsecs:         0


## Detección de colisiones del brazo robótico

El detector de colisiones debe ser capaz de identificar tanto auto-colisiones del brazo robótico (colisiones entre las distintas partes del brazo y el gripper) como colisiones con obstáculos definidos en el entorno. Consideraremos el gripper como un prisma rectangular para fines de detección de colisiones, usando las dimensiones específicas proporcionadas.

1. **Inicializar el Nodo ROS**: Comenzamos inicializando un nodo ROS. Este nodo facilitará la comunicación con otras partes del sistema ROS.

2. **Definición de Obstáculos**: A continuación, definimos los obstáculos presentes en el entorno que podrían interactuar con el brazo robótico. Cada obstáculo se define con un tipo (por ejemplo, 'cubo'), una pose inicial, dimensiones y un nombre único.


3. **Creación de una Instancia del Detector de Colisiones**: Instanciamos el `DetectorColisiones`, especificando si se incluirá el brazo robótico y/o el gripper, junto con los obstáculos definidos. 


4. **Verificación de Auto-colisiones y Colisiones con Obstáculos**: Utilizamos el método `hay_colision` del detector de colisiones para determinar si, dada una configuración específica de las articulaciones del brazo, existe alguna colisión con los obstáculos definidos o auto-colisiones.




In [10]:
import rospy
import manipulacion_lib  # Asegúrate de que este importe refleje la estructura real de tu proyecto

# 1. Inicializar el Nodo ROS
# Asegúrate de que el nodo solo se inicialice si aún no existe
if not rospy.get_node_uri():
    rospy.init_node('collision_detection', anonymous=True)

# 2. Definición de Obstáculos
# Crea una lista para almacenar los obstáculos definidos
obstaculos = []

# Crea y añade un obstáculo específico a la lista
# Asume que manipulacion_lib.Obstaculo es una clase que permite definir obstáculos con tipo, pose, dimensiones y nombre
obstaculo = manipulacion_lib.Obstaculo('cubo', [0.7, 0, 0, 1, 0, 0, 0], [0.8, 0.3, 0.7], 'obstaculo')
obstaculos.append(obstaculo)
obstaculos = []

# 3. Creación de una Instancia del Detector de Colisiones
# Inicializa el detector de colisiones con la configuración deseada, incluyendo el brazo robótico, el gripper y los obstáculos
detectorColisiones = manipulacion_lib.DetectorColisiones(
    usa_brazo_robotico=True,  # Indica si el brazo robótico se incluye en la detección de colisiones
    usa_gripper=True,  # Indica si el gripper se incluye en la detección de colisiones
    gripper_dimensions=[0.1, 0.1, 0.1],  # Especifica las dimensiones del gripper como un prisma rectangular
    obstaculos=obstaculos  # Lista de obstáculos definidos previamente
)

# 4. Verificación de Auto-colisiones y Colisiones con Obstáculos
# Define una configuración de ejemplo para las articulaciones del brazo robótico
configuracion_articulaciones = [0, 0, 0, 0, 0, 0]

# Utiliza el método hay_colision para verificar si la configuración actual del brazo robótico resulta en alguna colisión
hay_colision = detectorColisiones.hay_colision(configuracion_articulaciones)

# Imprime el resultado de la detección de colisiones
print(f"¿Hay colisión?: {'Sí' if hay_colision else 'No'}")

configuracion_articulaciones = [0, 0.05, 0, 0, 0, 0]
# Utiliza el método hay_colision para verificar si la configuración actual del brazo robótico resulta en alguna colisión
hay_colision = detectorColisiones.hay_colision(configuracion_articulaciones)

# Imprime el resultado de la detección de colisiones
print(f"¿Hay colisión?: {'Sí' if hay_colision else 'No'}")



usando gripper
obstaculo
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
num collision pairs - initial: 52
base_inertia:  1
shoulder:  2
upper:  3
forearm:  4
wrist1:  5
wrist2:  6
wrist3:  7
gripper:  8
suelo:  9
obstaculo :  10
¿Hay colisión?: No
¿Hay colisión?: Sí


# Transición de FakeRobot a GazeboRobot para Cálculos Cinemáticos

Al pasar de usar `FakeRobot` con RViz a `GazeboRobot` en Gazebo, el proceso para calcular la cinemática del brazo robótico es el mismo. Solo necesitas cambiar la instancia de `FakeRobot` por `GazeboRobot`. El resto del proceso y los métodos utilizados para calcular la cinemática son exactamente iguales.



In [5]:
import manipulacion_lib

# Cambia FakeRobot por GazeboRobot para simulaciones en Gazebo
ur10 = manipulacion_lib.GazeboRobot()

# El uso de la cinemática se mantiene igual
kin = manipulacion_lib.Cinematica(robot=ur10, frame_base='base_link', frame_final='tool0')

Unknown tag "scale" in /robot[@name='ur10']/link[@name='hand_finger_11_link']/collision[1]
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_knuckle_trans']/actuator[@name='hand_knuckle_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_12_trans']/actuator[@name='hand_finger_12_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_13_trans']/actuator[@name='hand_finger_13_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_22_trans']/actuator[@name='hand_finger_22_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_finger_23_trans']/actuator[@name='hand_finger_23_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name='ur10']/transmission[@name='hand_thumb_2_trans']/actuator[@name='hand_thumb_2_effort_motor']
Unknown tag "hardwareInterface" in /robot[@name